In [ ]:
import polars as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
from linknode_graph_patterns import LinkNodeGraphPatterns

def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_dfs_, _patterns_, _node_colors_ = [], LinkNodeGraphPatterns(), {}
for _pattern_ in _patterns_.types:
    _g_  = _patterns_.createPattern(_pattern_, prefix=_pattern_+'__')
    for _node_ in _g_.nodes: _node_colors_[_node_] = _pattern_
    _dfs_.append(graphToDataFrame(_g_))
df    = pl.concat(_dfs_)
g     = rt.createNetworkXGraph(df, [('fm','to')])
_pos_ = nx.spring_layout(g) # rtsvg.PolarsForceDirectedLayout(g).results()

In [ ]:
from math import sqrt

# separate g into different graph components
g_components              = [g.subgraph(c) for c in nx.connected_components(g)]
component_bounds, circles = [], []
for _g_ in g_components:
    # figure out the extents of the node positions
    x0, y0, x1, y1 = None, None, None, None
    for _node_ in _g_.nodes:
        x, y = _pos_[_node_]
        if x0 is None or x < x0: x0 = x
        if y0 is None or y < y0: y0 = y
        if x1 is None or x > x1: x1 = x
        if y1 is None or y > y1: y1 = y
    if x0 == x1: x0, x1 = x0 - 0.5, x1 + 0.5
    if y0 == y1: y0, y1 = y0 - 0.5, y1 + 0.5
    component_bounds.append((x0, y0, x1, y1))
    # add a circle to the center of the component
    _cx_, _cy_ = (x0+x1)*0.5, (y0+y1)*0.5
    _d_        = sqrt((x1-x0)**2 + (y1-y0)**2)
    circles.append((_cx_, _cy_, _d_/2.0))
_packed_ = rt.packCircles(circles)

# move the nodes of the component to the new positions
_new_pos_, _shapes_ = {}, {}
for i in range(len(g_components)):
    _g_ = g_components[i] 
    _dx_, _dy_ = circles[i][0] - _packed_[i][0], circles[i][1] - _packed_[i][1]
    for _node_ in _g_.nodes: _new_pos_[_node_] = _pos_[_node_][0] + _dx_, _pos_[_node_][1] + _dy_
    _shapes_[i] = f'<circle cx="{_packed_[i][0]}" cy="{_packed_[i][1]}" r="{_packed_[i][2]}" fill="#ffffff" stroke="#000000" stroke-width="1" />' # bg_shape_lu doesn't handle circles :(

_params_ = {'df':df, 'relationships':[('fm','to')], 'w':512, 'h':512, 'node_size':3, 'node_color':_node_colors_}
rt.tile([rt.link(**_params_, pos=_pos_),rt.link(**_params_, pos=_new_pos_, bg_shape_lu=_shapes_)])

In [ ]:
_df_  = pl.DataFrame({'fm':['a','b','c','d','e'], 'to':['b','a','d','e','c']})
_pos_ = {'a':(10.0,10.0), 'b':(15.0,20.0), 'c':(30.0,30.0), 'd':(40.0,40.0), 'e':(30.0,40.0)}
_bg_  = {'circle0':'<circle cx="12.5" cy="15.0" r="5"/>',
         'circle1':'<circle cx="30.0" cy="40.0" r="10.0"/>'}

rt.link(_df_, [('fm','to')], _pos_, bg_shape_lu=_bg_, w=512, h=128)
